In [30]:
import pandas as pd
import numpy as np
import xgboost
from xgboost import XGBClassifier, XGBRegressor
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, roc_auc_score, ConfusionMatrixDisplay, average_precision_score, median_absolute_error, mean_absolute_error
import matplotlib.pyplot as plt
import seaborn as sns
import tqdm
import econml.dml as dml
from sklearn.model_selection import train_test_split
from pandas import get_dummies

dml_causal = dml.CausalForestDML

df = pd.read_csv("../data/prepped_data.csv", index_col = 0, low_memory= False)
df = df[df['first_data_year'] >= 2021]
df = df.drop(columns=['policy_nr_hashed', 'last_data_year', 'first_data_year', 'control_group'])
print(df.columns)

Index(['welcome_discount', 'churn', 'first_premium', 'last_premium',
       'first_split', 'last_split', 'last_customer_age',
       'last_accident_free_years', 'last_car_value', 'last_age_car',
       'last_brand', 'last_type', 'last_weight', 'last_fuel_type',
       'last_postcode', 'last_product', 'last_allrisk basis',
       'last_allrisk compleet', 'last_allrisk royaal', 'last_wa-extra',
       'nr_cars', 'fake_alarm', 'policyholder_change', 'max_nr_coverages',
       'last_nr_coverages', 'last_trend_nr_coverages', 'accident_years',
       'last_year_car_change', 'last_change_premium_abs',
       'last_change_premium_perc', 'years_since_last_car_change',
       'n_last_vs_peak', 'last_vs_first_split', 'lpa',
       'cum_change_premium_abs', 'cum_change_premium_perc'],
      dtype='object')


In [31]:
categorical_features = []
continuous_features = []
binary_features = []

# Define a threshold for the maximum number of unique values for a categorical column
max_unique_values_for_categorical = 10

# Iterate through each column to determine if it's categorical, continuous, or binary
for column in df.columns:
    unique_values = df[column].nunique()
    if unique_values == 2:
        # If exactly 2 unique values, treat column as binary
        binary_features.append(column)
    elif (df[column].dtype == 'object' or unique_values <= max_unique_values_for_categorical) and unique_values > 2:
        # If object type or up to the threshold of unique values (and more than 2), treat as categorical
        categorical_features.append(column)
    else:
        # Otherwise, treat as continuous
        continuous_features.append(column)

categorical_features = [col for col in categorical_features if col != "years_since_last_car_change"]
continuous_features = continuous_features + ["years_since_last_car_change"]

print(f'Binary Features: {binary_features}')
print(f'Categorical Features: {categorical_features}')
print(f'Continuous Features: {continuous_features}')

df = pd.get_dummies(df, columns=categorical_features)




Binary Features: ['churn', 'last_allrisk basis', 'last_allrisk compleet', 'last_allrisk royaal', 'last_wa-extra', 'fake_alarm', 'policyholder_change', 'n_last_vs_peak', 'lpa']
Categorical Features: ['last_brand', 'last_type', 'last_fuel_type', 'last_postcode', 'last_product', 'nr_cars', 'max_nr_coverages', 'last_nr_coverages', 'last_trend_nr_coverages', 'last_year_car_change']
Continuous Features: ['welcome_discount', 'first_premium', 'last_premium', 'first_split', 'last_split', 'last_customer_age', 'last_accident_free_years', 'last_car_value', 'last_age_car', 'last_weight', 'accident_years', 'last_change_premium_abs', 'last_change_premium_perc', 'last_vs_first_split', 'cum_change_premium_abs', 'cum_change_premium_perc', 'years_since_last_car_change']


In [32]:
y = df['churn']
T = df['welcome_discount']
X = df.drop(columns=['churn', 'welcome_discount'])


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [33]:
best_first =  {'colsample_bytree': 0.6186073247946325, 'gamma': 0.16809946864097097, 'learning_rate': 0.1201301843520244, 'max_depth': 4.0, 'min_child_weight': 11.0, 'n_estimators': 141.0, 'reg_alpha': 0.9947688530022228, 'reg_lambda': 0.9032451749885752, 'subsample': 0.9954225737300325}

model_first = XGBClassifier(
    n_estimators=int(best_first['n_estimators']),
    max_depth=int(best_first['max_depth']),
    learning_rate=best_first['learning_rate'],
    subsample=best_first['subsample'],
    gamma=best_first['gamma'],
    colsample_bytree=best_first['colsample_bytree'],
    min_child_weight = best_first['min_child_weight'],
    # scale_pos_weight=scale_pos_weight_value,
    reg_alpha=best_first['reg_alpha'],
    reg_lambda=best_first['reg_lambda'],
    objective='binary:logistic',
    tree_method='hist',
    enable_categorical = True
)

# Fit the model
model_first.fit(X_train, y_train)

best_second = {'colsample_bytree': 0.8368650515536827, 'gamma': 0.0004689774751606146, 'learning_rate': 0.10657701450178336, 'max_depth': 8.0, 'min_child_weight': 10.0, 'n_estimators': 150.0, 'reg_alpha': 0.3708478046128461, 'reg_lambda': 0.6426896127366873, 'subsample': 0.96392422815337}


# Use the best parameters
model_second = XGBRegressor(
    n_estimators=int(best_second['n_estimators']),
    max_depth=int(best_second['max_depth']),
    learning_rate=best_second['learning_rate'],
    subsample=best_second['subsample'],
    gamma=best_second['gamma'],
    colsample_bytree=best_second['colsample_bytree'],
    min_child_weight = best_second['min_child_weight'],
    # scale_pos_weight=scale_pos_weight_value,
    reg_alpha=best_second['reg_alpha'],
    reg_lambda=best_second['reg_lambda'],
    objective='reg:squarederror',
    tree_method='hist',
    enable_categorical = True
)

# Fit the model
model_second.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.8368650515536827, device=None,
             early_stopping_rounds=None, enable_categorical=True,
             eval_metric=None, feature_types=None, gamma=0.0004689774751606146,
             grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.10657701450178336,
             max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=8, max_leaves=None,
             min_child_weight=10.0, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=150, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [35]:
cf = dml_causal(model_y=model_first,
                  model_t=model_second,
                  n_estimators=100,
                  # Other parameters as required
                  )

# Fit the model
cf.fit(y, T=T, X=X.dropna(axis=1), W=None)

# Estimate the causal effect
effects = cf.effect(X[:10])

print(effects)

ValueError: Input contains NaN.